# 03_data_load.ipynb

## 1. Sobre

Neste notebook vamos carregar o arquivo Parquet processado (`dados_projetos_limpos.parquet`) e persistir os dados em um banco PostgreSQL.

1) Antes de executar, certifique-se de que as **variáveis de conexão estão disponíveis** no ambiente (por exemplo via `.env`):

- db_host
- db_port
- db_name
- db_user
- db_passWORD

2) E de ter **levantado o container** com o banco de dados postegrees usando o comando:

`docker run --name meu-postgres -e POSTGRES_PASSWORD=minha_senha_secreta -e POSTGRES_USER=meu_usuario -e POSTGRES_DB=meu_banco -p 5432:5432 -d postgres`

3) Também verifique se as dependências estão instaladas: `sqlalchemy`, `psycopg2-binary` e `pandas`.

In [26]:
import os
import sys
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

project_root = Path(os.getcwd()).resolve().parents[0]
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Carrega .env
load_dotenv(override=True)


True

## 2. Carregamento dos Dados

Nessa célula está implementado o **carregamento dos dados limpos** gerados em `02_data_transform.ipynb`. Os quais, estão armazenados na pasta `data/processed/` em parquet, visando **comportabilidade** e **persistencia de tipagem**.

In [8]:
ROOT = Path(os.getcwd()).resolve().parents[0]
PROCESSED_PATH = ROOT / 'data' / 'processed'
print('Parquet file:', PROCESSED_PATH)

# Lê o parquet
print("\nDataFrame de Projetos:")
df_projetos = pd.read_parquet(str(PROCESSED_PATH) + '/dados_projetos_limpos.parquet')
print('Loaded rows:', len(df_projetos))
print(df_projetos.dtypes)

print("\nDataFrame de Executores:")
df_executores = pd.read_parquet(str(PROCESSED_PATH) + '/dados_executores_limpos.parquet')
print('Loaded rows:', len(df_executores))
print(df_executores.dtypes)

print("\nDataFrame de Eixos:")
df_eixos = pd.read_parquet(str(PROCESSED_PATH) + '/dados_eixos_limpos.parquet')
print('Loaded rows:', len(df_eixos))
print(df_eixos.dtypes)

print("\nDataFrame de Subtipos:")
df_subtipos = pd.read_parquet(str(PROCESSED_PATH) + '/dados_subtipos_limpos.parquet')
print('Loaded rows:', len(df_subtipos))
print(df_subtipos.dtypes)

print("\nDataFrame de Fontes de Recurso:")
df_fontes_recurso = pd.read_parquet(str(PROCESSED_PATH) + '/dados_fontes_recurso_limpos.parquet')
print('Loaded rows:', len(df_fontes_recurso))
print(df_fontes_recurso.dtypes)


Parquet file: E:\VS-Code\Lappis-PS\data\processed

DataFrame de Projetos:
Loaded rows: 800
id_unico                                  string[python]
nome                                      string[python]
cep                                       string[python]
endereco                                  string[python]
descricao                                 string[python]
funcao_social                             string[python]
meta_global                               string[python]
data_inicial_prevista                     datetime64[ns]
data_final_prevista                       datetime64[ns]
data_inicial_efetiva                      datetime64[ns]
data_final_efetiva                        datetime64[ns]
data_cadastro                             datetime64[ns]
especie                                   string[python]
natureza                                  string[python]
natureza_outras                           string[python]
situacao                                  string[pytho

## 3. Conexão e gravação no BD

Utilizando o banco de dados **postgres** em um container **docker**. 
- Associação das **credências** dentro do **.env** na raiz com o banco de dados.
- **Carregamento do banco de dados** com os DataFrames.

In [23]:
# Configura a conexão com PostgreSQL a partir das variáveis de ambiente
db_host = os.getenv('PG_HOST')
db_port = os.getenv('PG_PORT', '5432')  # porta padrão do PostgreSQL
db_name = os.getenv('PG_DB')
db_user = os.getenv('PG_USER')
db_pass = os.getenv('PG_PASSWORD')

if not all([db_host, db_name, db_user, db_pass]):
    raise RuntimeError('Variáveis de conexão PostgreSQL não estão definidas. Configure db_host/db_name/db_user/db_passWORD no .env')

# cria engine SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')
print('Engine created')

# itera sobre os dataframes e escreve cada um como uma tabela no banco de dados
for df, name in [(df_projetos, 'dados_projetos'), (df_executores, 'dados_executores'),
                 (df_eixos, 'dados_eixos'), (df_subtipos, 'dados_subtipos'),
                 (df_fontes_recurso, 'dados_fontes_recurso')]:
    print(f'Writing table: {name} with {len(df)} rows')
    df.to_sql(name, engine, if_exists='replace', index=False, method='multi', chunksize=1000)
    print('Table written:', name)


Engine created
Writing table: dados_projetos with 800 rows
Table written: dados_projetos
Writing table: dados_executores with 875 rows
Table written: dados_executores
Writing table: dados_eixos with 967 rows
Table written: dados_eixos
Writing table: dados_subtipos with 967 rows
Table written: dados_subtipos
Writing table: dados_fontes_recurso with 800 rows
Table written: dados_fontes_recurso


## 4. Teste Unitário (Simples)

Teste Unitário de **Query**.

In [ ]:
# Validação rapida
with engine.connect() as conn:
    # Crie o objeto de texto SQL
    query_count = text('SELECT COUNT(*) FROM dados_projetos')
    
    # Execute o objeto
    count = conn.execute(query_count).scalar()
    print('Row count in DB table:', count)
    
    # Faça o mesmo para a query do pandas
    query_select = text('SELECT * FROM dados_projetos LIMIT 5')
    sample = pd.read_sql(query_select, conn)
    
    print("\nAmostra dos dados no banco:")
    print(sample.head())

Row count in DB table: 800

Amostra dos dados no banco:
      id_unico                                               nome         cep  \
0  50379.53-54  DL - 304/2024 - Contratação de instituição par...        None   
1  42724.53-27                  Escola Classe Crixá São Sebastião        None   
2  19970.53-78  Reajuste do Contrato 45/2021 - Contrução do Ce...  70.602-600   
3  24797.53-15  Implantação de Passarelas nas Estradas Parque ...        None   
4  24822.53-70  obra de construção da  Cabine de Medição, loca...        None   

                                   endereco  \
0                                      None   
1                                      None   
2  SAIS Área Especial 3, Setor Policial Sul   
3                                      None   
4                                      None   

                                           descricao  \
0  Contratação de instituição para execução de se...   
1  Construção de Escola em Tempo Integral, Escola...   
2  Rea